# LSM CEF Equation Of State Fitting 

## Generating the CEF equations to fit for LSM

In [1]:
# General modules and settings


scan_alpha = False # Whether to do hyperparameter optimization for alpha

In [2]:
# Generate the CEF equations for LSM
from HydroGEN_CEF_general_current import *

# Set the lattice string for LSM
lattice='(La,Sr,Va)1(Mn,Va)1(O,Va)3'

# Generate the CEF equations for fitting Gibbs energy without configurational entropy
# These expressions are currently being saved to the file CEF_LaMnO
generate_cef_module(lattice,G_neutral_values=[],T_terms=[1,T,T*log(T)],ir=(-1,-1),use_neutral=False)

No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of conditions!
No "exact" solution for this combination of cond

## Read in and parse the CEF expressions

In [3]:
# Read in relevant modules and set some global parameters
import re
import os
import numpy as np
import math
from shutil import copyfile,rmtree
import copy
from sympy import symbols, solve, simplify, sympify, preorder_traversal, Float, log, diff, apart
import itertools
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import fmin_l_bfgs_b
r = 8.617333262E-5; # Boltzman constant in eV/K.
R=8.314 # gas constant in J / mol·K
emax=2.99 # highest O stoichiometry used for fitting.
color_table=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']+['xkcd:salmon','xkcd:light green','xkcd:yellow'
    ,'xkcd:navy','xkcd:light pink','xkcd:gold','xkcd:neon pink','xkcd:baby blue','xkcd:forrest green'
    ,'xkcd:cobalt blue','xkcd:reddish pink','xkcd:hot purple','xkcd:ecru','xkcd:rust red'
    ,'xkcd:fern green','xkcd:rose red']

y0_values=[0,0.1,0.2,0.3,0.4,0.5]

In [4]:
# Read Gibbs Energy expression in from CEF_LaMnO file
cef_results='CEF_LaMnO'
with open(cef_results,'r') as f:
	c=f.readlines()

for i in range(len(c)):
	if 'Nonconfigurational-Gibbs energy with T expansion:' in c[i]:
		str_G_ncf=c[i+1].strip('\n')
	if 'Total Gibbs energy with T expansion:' in c[i]:
		str_G=c[i+1].strip('\n')
	if 'Independent site fractions:' in c[i]:
		str_y=c[i+1].strip('\n')
	if 'Endmember parameters:' in c[i]:
		str_a=c[i+1].strip('\n')
	if 'Independent excess parameters:' in c[i]:
		str_L=c[i+1].strip('\n')

sy=str_y.strip('[').strip(']').split(', ')
y=[symbols('y'+str(i)) for i in range(len(sy))]
a=symbols(str_a.strip('[').strip(']').replace(',',' '))
L=symbols(str_L.strip('[').strip(']').replace(',',' '))
T=symbols('T')

dreplace={}
for i in range(len(sy)):
	dreplace[sy[i]]=str(y[i])

for key,value in dreplace.items():
	str_G_ncf=str_G_ncf.replace(key,value)
	str_G=str_G.replace(key,value)

G_ncf=sympify(str_G_ncf)
G=sympify(str_G)

# fix to reference perfect compound T-dependence
G=G.subs(symbols('a_0_1'),0).subs(symbols('a_0_2'),0)
G_ncf=G_ncf.subs(symbols('a_0_1'),0).subs(symbols('a_0_2'),0)

G_ncf_0K=G_ncf.subs(log(T),0).subs(T,0)
G_ncf_0K=simplify(G_ncf_0K.expand())

In [5]:
# Read in data files

# read 0K DFT data
data=np.loadtxt('data')
data_0K=data[data[:,-2]==0]

# lasso feature selection
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.datasets import load_diabetes

L_features=[diff(G_ncf_0K,i) for i in L]
a_features=[diff(G_ncf_0K,i) for i in a]

X=[]
f=[]
for i in range(data_0K.shape[0]):
	y_values=data_0K[i,:][0:len(y)]
	d_y_subs={}
	for j in range(len(y)):
		d_y_subs[y[j]]=y_values[j]
	L_features_values=[j.subs(d_y_subs) for j in L_features]
	a_features_values=[j.subs(d_y_subs) for j in a_features]
	X.append(L_features_values+a_features_values)
	f.append(data_0K[i,:][-1])

# read O2 Gibbs energy
mu_ft=np.loadtxt('O2_janaf')
T_O2=mu_ft[:,0]
S_O2=mu_ft[:,2]
H_O2=mu_ft[:,4]
G_O2=H_O2*1000-T_O2*S_O2 # J/mol O2
G_O2=G_O2/(1.6e-19)/(6.02e23) # eV per O2
G_O2=G_O2-G_O2[0]
G_O=G_O2/2

from scipy import interpolate
G_O_ip=interpolate.interp1d(T_O2, G_O)

for n in range(len(y0_values)):
    if y0_values[n]!=0:
        Ga_op=G.subs({y[1]*log(y[1]):0,y[2]*log(y[2]):0}).subs({y[0]:y0_values[n],y[1]:0,y[2]:0})
    else:
        Ga_op=G.subs({y[0]*log(y[0]):0,y[1]*log(y[1]):0,y[2]*log(y[2]):0}).subs({y[0]:0,y[1]:0,y[2]:0})
    mu_O_op=-diff(Ga_op,y[-1])/3
    mu_O_op=simplify(mu_O_op.expand())
    L_features_op=[diff(mu_O_op,i) for i in L]
    a_features_op=[diff(mu_O_op,i) for i in a]
    d_zeros={}
    for i in L:
        d_zeros[i]=0
    for i in a:
        d_zeros[i]=0
    c_op=mu_O_op.subs(d_zeros) # terms without L and a
    import glob
    fexpt0=glob.glob('expt_*')
    fexpt=[]
    for j in range(len(fexpt0)):
        with open(fexpt0[j],'r') as files:
            if float(files.readlines()[0].strip('#'))==y0_values[n]:
                fexpt.append(fexpt0[j])
    expt=[np.loadtxt(i) for i in fexpt]
    for j in range(len(expt)):
        expt_current=expt[j][expt[j][:,1]<emax]
        for i in range(expt_current.shape[0]):
            d_subs={T:expt_current[i,2],y[-1]:(3-expt_current[i,1])/3}
            L_features_op_values=[k.subs(d_subs) for k in L_features_op]
            a_features_op_values=[k.subs(d_subs) for k in a_features_op]
            X.append(L_features_op_values+a_features_op_values)        
            mu=1/2*r*expt_current[i,2]*np.log(expt_current[i,0])+G_O_ip(expt_current[i,2])
            f.append(mu-c_op.subs(d_subs)) 

X=np.array(X)
f=np.array(f)
scaler = StandardScaler(with_mean=False)
scaler.fit(X)
X_st=scaler.transform(X)

features = [str(i) for i in L]+[str(i) for i in a]

## Determine fitting hyperparameters
Do a grid scan to determine a good value for alpha in the LASSO fitting approach

In [6]:
if scan_alpha: 

    # Runs to gather data for alpha-importance diagram
    clf = Lasso(alpha=1e-11,tol=1e-13,max_iter=int(1e7),fit_intercept=False)
    clf.fit(X_st, f)
    importance0 = np.abs(clf.coef_)

    log_alpha=[-11+0.2*i for i in range(56)]
    importances=[]
    for i in log_alpha:
        clf = Lasso(alpha=10**(i),tol=1e-13,max_iter=int(1e7),fit_intercept=False)
        clf.fit(X_st, f)
        importance = np.abs(clf.coef_)
        importances.append(importance)

    importances=np.array(importances)
    log_alpha=np.array(log_alpha)
    log_alpha=log_alpha.reshape(-1,1)

    # plot alpha-importance diagram
    ia,iL=0,0
    for i in range(importances[:,importance0 > 0].shape[1]):
        label=np.array(features)[importance0 > 0][i]
        if 'L' in label:
            linestyle="dashdot"
            iL=iL+1;icolor=iL
        else:
            linestyle="solid"
            ia=ia+1;icolor=ia
        plt.plot(log_alpha,importances[:,importance0 > 0][:,i],label=label,linestyle=linestyle,color=color_table[icolor])

    plt.xlabel("$\mathregular{logα}$",fontsize=15)
    plt.ylabel('Importance',fontsize=15)
    plt.subplots_adjust(left=0.07, right=0.8, bottom=0.1, top=0.93)
    plt.legend(bbox_to_anchor=(1.01, 0.9), loc="upper left",frameon=False,ncol=2,columnspacing=1)
    plt.savefig("alpha_importance.pdf")


<>:32: SyntaxWarning: invalid escape sequence '\m'
<>:32: SyntaxWarning: invalid escape sequence '\m'
/var/folders/ng/25b6m7n90c78bk8x_235kxj4000gdz/T/ipykernel_61163/4222569589.py:32: SyntaxWarning: invalid escape sequence '\m'
  plt.xlabel("$\mathregular{logα}$",fontsize=15)
